In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-126764")
exp = Experiment(workspace=ws, name="quick-starts-ws-126764")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FLQEK7MCQ to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-126764
Azure region: southcentralus
Subscription id: 174c6bee-3e04-4ee5-98ea-6d411844e6dd
Resource group: aml-quickstarts-126764


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cluster_name = 'Test'
provisioning_config = AmlCompute.provisioning_configuration(
        vm_size='Standard_D2_v2',        
        max_nodes=4)

if cluster_name in ws.compute_targets:
    compute_target = ws.compute_targets[cluster_name]
    if type(compute_target) is not AmlCompute:
        raise Exception('Compute target {} is not an AML cluster.'
                        .format(cluster_name))
    print('Using pre-existing AML cluster {}'.format(cluster_name))
else:
    # Create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, provisioning_config)

    # You can poll for a minimum number of nodes and set a specific timeout. 
    # If min node count is provided, provisioning will use the scale settings for the cluster.
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [36]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--max_iter': uniform(50,200),
    '--C': uniform(0.2,0.8)
})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=50, slack_factor=0.2, delay_evaluation=200)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(".", compute_target=compute_target, entry_script="train1.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    estimator=est,
    hyperparameter_sampling=ps,
    policy=policy,
    max_total_runs=20,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE
    )

In [22]:
os.listdir()

['.ipynb_aml_checkpoints',
 'outputs',
 'train1.py',
 'training',
 'udacity-project (1).ipynb']

In [37]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

run = exp.submit(hyperdrive_config)
RunDetails(run).show()
run


WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

Experiment,Id,Type,Status,Details Page,Docs Page
quick-starts-ws-126764,HD_b207e17a-c276-41a3-9ca5-8638c0e4499a,hyperdrive,Running,Link to Azure Machine Learning studio,Link to Documentation


In [35]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = run.get_best_run_by_primary_metric()
best_run.download_file("outputs/model.joblib", "./outputs/model.joblib")
model = best_run.register_model(model_name="best_run_model", model_path="outputs/model.joblib")


TypeError: can't pickle _thread.RLock objects

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(### YOUR DATA OBJECT HERE ###)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

### YOUR CODE HERE ###

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###